# Lecture 36: Case Study

---

## Overview

...

The Adaptive Large Neighbourhood Search (ALNS) algorithm begins by setting the current solution – $s$, and the best solution – $s^*$, to the initial solution – $s_o$. In addition, it also initialises a long-term memory of previously visited solutions in the form of a hash list to prevent redundant search. Further, given a set of removal operators – $o_r$, and insertion operators – $o_i$, the algorithm assigns unit weight to each operator, thus ensuring unbiased search at the outset. Within this adaptive large neighbourhood search procedure, the algorithm also runs a simulated annealing mechanism to enable the algorithm to comprehensively explore the solution landscape. In the ALNS implementation here, the simulated annealing mechanism begins search with a temperature that allows the algorithm to accept a solution up to $\overline{\omega}$ times worse than the initial solution with a probability $\overline{\tau}$. 

Hereafter, the ALNS algorithm runs $j$ segments, each comprising of $n$ iterations. A segment begins with an update of operator probabilities based on the recent updates to operator weights, as well as a reset of operator counts and scores to 0. Thereafter, within each iteration of the segment, the ALNS algorithm randomly selects a removal and an insertion operator based on operator selection probabilities, and subsequently, updates their counts. It then deploys the removal and insertion operator on the current solution, thus destroying and rebuilding the solution to generate a new solution. Note, the size of these operations is stochastic and contingent on thresholds – $\underline{\mu}$,$\overline{\mu}$.

Based on the objective function evaluations of the solutions – $f$; If the resulting new solution is better than the best solution, then the ALNS algorithm updates the current and the best solution, and subsequently, rewards the operators with a score of $\sigma_1$. However, if the new solution in only better than the current solution, then the algorithm updates only the current solution. Further, the operators receive a relatively lower score of $\sigma_2$, yet only if the new solution is novel. Nonetheless, if the new solution is worse than the current solution, then the ALNS algorithm accepts it as the current solution with a small probability, determined by the Boltzmann function – $p=\text{exp}⁡((f(s)-f(s' ))/T_k)$. For this update, the operators receive an even lower score of $\sigma_3$, but yet again, only if the new solution is novel. 

Finally, at the end of each iteration, the ALNS algorithm stores the current solution into the hash list and updates the search temperature based on the cooling schedule. In the implementation here, the search cools down at an exponential rate dictated by cooling factor - $\varphi$, nonetheless, the algorithm always maintains a minimum temperature that enables the algorithm to accept a solution up to $\underline{\omega}$ times worse than the initial solution with a probability $\underline{\tau}$. 

After completing $k$ such segments, the algorithm resets current solution to the best solution, thus reinforcing the focus on high-quality solutions. At the end of each segment, the ALNS algorithm updates operator weights based on operator performance, i.e., operator scores normalised by operator counts. In particular, the algorithm reacts to the operator performance in the current segment through a reaction factor – $\rho$, while accounting for operator performance in previous segments through a dissipation factor – $(1-\rho)$. Further, before initiating the next segment, the algorithm refines the current solution through local search operators – $o_l$, applied iteratively, each for $m$ iterations. 

Finally, upon completing j×n iterations, the algorithm returns the best solution.

---

## Pseudo Code

1. **Procedure** $\text{ALNS}(s_o, (j, k, n, m, o_r, o_i, o_l, \sigma_1, \sigma_2, \sigma_3, \underline{\mu}, \overline{\mu}, \underline{\omega}, \overline{\omega}, \underline{\tau}, \overline{\tau}, \varphi, \rho))$
2. $s ← s_o$ &emsp;<small>// initialise current solution $s$ as the initial solution $s_o$</small>
3. $s^* ← s$ &emsp;<small>// initialise best solution $s^*$ as the current solution</small>
4. $T ← \overline{\omega} f(s^*) / \ln(1 / \overline{\tau})$ &emsp;<small>// initialise temperature based on cooling schedule</small>
5. $H ← \{h(s)\}$ &emsp;<small>// initialise hash list</small>
6. **for** $o_r ∈ o_r$ **do** &emsp;<small>// set removal operator weights to 1</small>
7. &emsp;$w_r ← 1$
8. **end for**
9. **for** $o_i ∈ o_i$ **do** &emsp;<small>// set insertion operator weights to 1</small>
10. &emsp;$w_i ← 1$
11. **end for**
12. $u ← 1$ &emsp;<small>// set segment index to 1</small>
13. **while** $u ≤ j$ **do** &emsp;<small>// repeat for $j$ segments</small>
14. &emsp;**for** $o_r ∈ o_r$ **do**
15. &emsp;&emsp;$c_r ← 0$ &emsp;<small>// set removal operator count to 0</small>
16. &emsp;&emsp;$\pi_r ← 0$ &emsp;<small>// set removal operator score to 0</small>
17. &emsp;&emsp;$p_r ← w_r / \sum_{r ∈ Ψ_r} w_r$ &emsp;<small>// update removal operator probability</small>
18. &emsp;**end for**
19. &emsp;**for** $o_i ∈ o_i$ **do**
20. &emsp;&emsp;$c_i ← 0$ &emsp;<small>// set insertion operator count to 0</small>
21. &emsp;&emsp;$\pi_i ← 0$ &emsp;<small>// set insertion operator score to 0</small>
22. &emsp;&emsp;$p_i ← w_i / \sum_{i ∈ Ψ_i} w_i$ &emsp;<small>// update insertion operator probability</small>
23. &emsp;**end for**
24. &emsp;$v ← 1$ &emsp;<small>// set iteration index to 1</small>
25. &emsp;**while** $v ≤ n$ **do** &emsp;<small>// repeat for $n$ iterations</small>
26. &emsp;&emsp;$o_r {R \atop ←} p_r$ &emsp;<small>// randomly select a removal operator</small>
27. &emsp;&emsp;$o_i {R \atop ←} p_i$ &emsp;<small>// randomly select an insertion operator</small>
28. &emsp;&emsp;$c_r ← c_r + 1$ &emsp;<small>// update removal operator count</small>
29. &emsp;&emsp;$c_i ← c_i + 1$ &emsp;<small>// update insertion operator count</small>
30. &emsp;&emsp;$\Lambda \sim U(0, 1)$
31. &emsp;&emsp;$\lambda {R \atop ←} \Lambda$
32. &emsp;&emsp;$q ← [(1 - \lambda) \underline{\mu} ||s|| + \lambda \overline{\mu} ||s||]^-$  &emsp;<small>// set the size of removal/insertion operation</small>
33. &emsp;&emsp;$s' ← o_i(o_r(q, s))$ &emsp;<small>// remove and insert selected customer nodes</small>
34. &emsp;&emsp;**if** $f(s') < f(s^*)$ **then** &emsp;<small>// if the new solution is better than the best solution</small>
35. &emsp;&emsp;&emsp;$s^* ← s'$ &emsp;<small>// update the best solution</small>
36. &emsp;&emsp;&emsp;$s ← s'$ &emsp;<small>// update the current solution</small>
37. &emsp;&emsp;&emsp;$\pi_r ← \pi_r + \sigma_1$ &emsp;<small>// update removal operator score by $\sigma_1$</small>
38. &emsp;&emsp;&emsp;$\pi_i ← \pi_i + \sigma_1$ &emsp;<small>// update insertion operator score by $\sigma_1$</small>
39. &emsp;&emsp;**else if** $f(s') < f(s)$ **then** &emsp;<small>// else if the new solution is better than the current solution</small>
40. &emsp;&emsp;&emsp;$s ← s'$
41. &emsp;&emsp;&emsp;**if** $h(s) ∉ H$ **then** &emsp;<small>// if the solution does not exist in the hashed tabu list</small>
42. &emsp;&emsp;&emsp;&emsp;$\pi_r ← \pi_r + \sigma_2$ &emsp;<small>// update removal operator score by $\sigma_2$</small>
43. &emsp;&emsp;&emsp;&emsp;$\pi_i ← \pi_i + \sigma_2$ &emsp;<small>// update insertion operator score by $\sigma_2$</small>
44. &emsp;&emsp;&emsp;**end if**
45. &emsp;&emsp;**else** &emsp;<small>// else accept new solution with a small probability</small>
46. &emsp;&emsp;&emsp;$\Lambda \sim U(0, 1)$
47. &emsp;&emsp;&emsp;$\lambda {R \atop ←} \Lambda$
48. &emsp;&emsp;&emsp;**if** $\lambda < \exp((f(s) - f(s')) / T)$ **then** 
49. &emsp;&emsp;&emsp;&emsp;$s ← s'$ &emsp;<small>// update current solution</small>
50. &emsp;&emsp;&emsp;&emsp;**if** $h(s) ∉ H$ **then**
51. &emsp;&emsp;&emsp;&emsp;&emsp;$\pi_r ← \pi_r + \sigma_3$ &emsp;<small>// update removal operator score by $\sigma_3$</small>
52. &emsp;&emsp;&emsp;&emsp;&emsp;$\pi_i ← \pi_i + \sigma_3$ &emsp;<small>// update insertion operator score by $\sigma_3$</small>
53. &emsp;&emsp;&emsp;&emsp;**end if**
54. &emsp;&emsp;&emsp;**end if**
55. &emsp;&emsp;**end if**
56. &emsp;&emsp;$H ← H ∪ \{h(s)\}$ &emsp;<small>// add current solution to hash list</small>
57. &emsp;&emsp;$T ← \max(\phi T, \underline{\omega} f(s^*) / \ln(1 / \underline{\tau}))$ &emsp;<small>// update temperature based on the cooling schedule</small>
58. &emsp;&emsp;$v ← v + 1$ &emsp;<small>// update iteration index</small>
59. &emsp;**end while**
60. &emsp;**for** $o_r ∈ o_r$ **do** &emsp;<small>// update removal operator weights</small>
61. &emsp;&emsp;**if** $c_r ≠ 0$ **then**
62. &emsp;&emsp;&emsp;$w_r ← \rho \pi_r / c_r + (1 - \rho) w_r$
63. &emsp;&emsp;**end if**
64. &emsp;**end for**
65. &emsp;**for** $o_i ∈ o_i$ **do** &emsp;<small>// update insertion operator weights</small>
66. &emsp;&emsp;**if** $c_i ≠ 0$ **then**
67. &emsp;&emsp;&emsp;$w_i ← \rho \pi_i / c_i + (1 - \rho) w_i$
68. &emsp;&emsp;**end if**
69. &emsp;**end for**
70. &emsp;**if** $h \mod k$ **then** &emsp;<small>// after every $k$ segments, reset the current solution to the best solution</small>
71. &emsp;&emsp;$s ← s^*$
72. &emsp;**end if**
73. &emsp;**for** $o_l ∈ o_l$ &emsp;<small>// iteratively perform local search on the current solution</small>
74. &emsp;&emsp;$s ← o_l(s, m)$
75. &emsp;**end for**
76. &emsp;**if** $f(s) < f(s^*)$ **then** &emsp;<small>// if the current solution is better than the best solution</small>
77. &emsp;&emsp;$s^* ← s$ &emsp;<small>// update the best solution to the current solution</small>
78. &emsp;**end if**
79. &emsp;$H ← H ∪ \{h(s)\}$ &emsp;<small>// add the current solution to the hash list</small>
80. &emsp;$u ← u + 1$ &emsp;<small>// update segment index</small>
81. **end while**
82. **return** $s^*$ &emsp;<small>// return the best solution</small>

---

## Implementation

---

## Case Study

---

```{note}
...
```
